In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
import warnings
import pickle
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
warnings.filterwarnings("ignore")

# Veriyi yükleme
data = pd.read_csv('diabetes.csv', encoding='utf-8')

# Eksik değerlerin işlenmesi
columns_with_zeros = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
data[columns_with_zeros] = data[columns_with_zeros].replace(0, pd.NA)

for col in columns_with_zeros:
    data.loc[data['Outcome'] == 0, col] = data.loc[data['Outcome'] == 0, col].fillna(data.loc[data['Outcome'] == 0, col].mean())
    data.loc[data['Outcome'] == 1, col] = data.loc[data['Outcome'] == 1, col].fillna(data.loc[data['Outcome'] == 1, col].mean())

# Özellik ve etiketleri ayırma
X = data.drop('Outcome', axis=1)
y = data['Outcome']

# Veri dengesizliğini giderme
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Özellikleri standartlaştırma
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_resampled)

# Eğitim ve test verilerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

xgb_model = XGBClassifier(
    random_state=42, 
    use_label_encoder=False, 
    eval_metric='logloss'
)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)
print("Doğruluk Skoru:", accuracy_score(y_test, y_pred))


# Modeli ve scaler'ı pickle ile kaydet
with open('xgb_model97.pkl', 'wb') as model_file:
    pickle.dump(xgb_model, model_file)

with open('scaler97.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

print("Model ve scaler başarıyla kaydedildi.")

# Doğruluk, kesinlik, duyarlılık ve F1 skoru
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Doğruluk Skoru:", accuracy)
print("Kesinlik (Precision):", precision)
print("Duyarlılık (Recall):", recall)
print("F1 Skoru:", f1)
print("\nClassification Report:\n", classification_rep)



Doğruluk Skoru: 0.915
Model ve scaler başarıyla kaydedildi.
Doğruluk Skoru: 0.915
Kesinlik (Precision): 0.8952380952380953
Duyarlılık (Recall): 0.94
F1 Skoru: 0.9170731707317075

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.89      0.91       100
           1       0.90      0.94      0.92       100

    accuracy                           0.92       200
   macro avg       0.92      0.92      0.91       200
weighted avg       0.92      0.92      0.91       200



In [2]:
print(data.head())

   Pregnancies  Glucose  BloodPressure  SkinThickness     Insulin   BMI  \
0            6    148.0           72.0           35.0  206.846154  33.6   
1            1     85.0           66.0           29.0  130.287879  26.6   
2            8    183.0           64.0           33.0  206.846154  23.3   
3            1     89.0           66.0           23.0   94.000000  28.1   
4            0    137.0           40.0           35.0  168.000000  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  


In [42]:
# Keras modelini oluşturuyoruz, 8 özellik ile
keras_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X.shape[1],)),  # 8 özellik
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Keras modelini derleyin
keras_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Keras modelini eğitme
keras_model.fit(X, y, epochs=10, batch_size=32)

# Keras modelini kaydedin
keras_model.save('kerass_model.h5')


Epoch 1/10
24/24 [==============================] - 3s 10ms/step - loss: 1.6357 - accuracy: 0.6576
Epoch 2/10
24/24 [==============================] - 0s 8ms/step - loss: 0.5492 - accuracy: 0.7604
Epoch 3/10
24/24 [==============================] - 0s 9ms/step - loss: 0.5503 - accuracy: 0.7474
Epoch 4/10
24/24 [==============================] - 0s 8ms/step - loss: 0.5345 - accuracy: 0.7682
Epoch 5/10
24/24 [==============================] - 0s 8ms/step - loss: 0.5525 - accuracy: 0.7786: 0s - loss: 0.6152 - accuracy: 
Epoch 6/10
24/24 [==============================] - 0s 8ms/step - loss: 0.5243 - accuracy: 0.7734
Epoch 7/10
24/24 [==============================] - 0s 8ms/step - loss: 0.5349 - accuracy: 0.7734
Epoch 8/10
24/24 [==============================] - 0s 10ms/step - loss: 0.4987 - accuracy: 0.7852
Epoch 9/10
24/24 [==============================] - 0s 10ms/step - loss: 0.5296 - accuracy: 0.7708
Epoch 10/10
24/24 [==============================] - 0s 8ms/step - loss: 0.5066 - a

In [43]:
# Keras modelini yükle
keras_model = tf.keras.models.load_model('kerass_model.h5')

# Modeli TensorFlow Lite formatına dönüştür


converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
tflite_model = converter.convert()

# TFLite modelini kaydedin
with open('model97.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model başarıyla TFLite formatına dönüştürüldü ve kaydedildi.")


INFO:tensorflow:Assets written to: C:\Users\Asus\AppData\Local\Temp\tmpito5d69t\assets


INFO:tensorflow:Assets written to: C:\Users\Asus\AppData\Local\Temp\tmpito5d69t\assets


Model başarıyla TFLite formatına dönüştürüldü ve kaydedildi.


In [29]:
# Outcome sütununun dağılımını kontrol et
print(data['Outcome'].value_counts())

0    500
1    268
Name: Outcome, dtype: int64


In [ ]:
# Tahmin yapmak için input verisi
#input_query = np.array([[0, 120, 75, 28, 110.4, 27.5, 0.500, 35]])
#input_query = np.array([[6, 148, 72, 35, 206.84, 33.6, 0.627, 50]])
#input_query = np.array([[1, 89, 66, 23, 94, 28.1, 0.167, 21]])
#input_query = np.array([[0, 137, 40, 35, 168,43.1, 2.288, 33]])
#input_query = np.array([[1, 85, 66, 29, 130.28,26.6, 0.351, 31]])
input_query = np.array([[0, 72, 47, 25, 148,30.1, 0.128, 28]])
# Input verisini ölçeklendir
input_query_scaled = scaler.transform(input_query)
# Model ile tahmin yap
prediction = xgb_model.predict(input_query_scaled)
prediction_proba =xgb_model.predict_proba(input_query_scaled)[:, 1]

# Tahmin ve olasılıkları yazdır
print(f"Tahmin Sonucu (0 veya 1): {prediction[0]}")
print(f"Diyabet olma olasılığı: {prediction_proba[0]:.4f}")
